In [1]:
# %pip install tensorflow
%pip install scikit-learn


  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 4.7 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [39]:
import cv2
from keras._tf_keras.keras.optimizers import Adam
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd

In [4]:
words = np.array([
    'หนังสือ', 'ทารก', 'อยู่', 'อาหาร', 'พูด', 'คน', 'ยินดีด้วย', 'ผู้หญิง',
    'วันอังคาร', 'วันเสาร์', 'ต้นไม้', 'ทำไม', 'อ่าน', 'ฟัง', 'ภาษามือไทย', 'กระโดด',
    'จาน', 'วันศุกร์', 'แมว', 'ดื่ม', 'คุณ', 'ผลไม้', 'กลางคืน', 'ช้อน',
    'ร้องไห้', 'ตื่น', 'ดอกไม้', 'ของขวัญ', 'คอมพิวเตอร์', 'สัตว์', 'วันพฤหัสบดี', 'วันจันทร์',
    'ยืน', 'เล่น', 'ยิ้ม', 'ไม่ใช่', 'ทะเล', 'เขา', 'หมา', 'กิน', 'โรงเรียน',
    'ฉัน', 'เพื่อน', 'ที่ไหน', 'เด็ก', 'ภาษาไทย', 'บ้าน', 'นม', 'ปิด', 'วันอาทิตย์',
    'ไม่เป็นไร', 'ขอบคุณ', 'เปิด', 'ลาก่อน', 'ทำ', 'ผู้ชาย', 'ภูเขา', 'ใคร', 'ท้องฟ้า',
    'เธอ', 'ครอบครัว', 'เขียน', 'สบายดี', 'เดิน', 'นั่ง', 'วันพุธ', 'แตงโม', 'ล้าง', 'อย่างไร'
])


In [27]:
words = np.array(os.listdir('Landmarked'))

In [43]:
DATA_PATH = os.path.join('Landmarked')

sequence_length = 30
no_sequences = 1

In [29]:
from sklearn.model_selection import train_test_split
from keras._tf_keras.keras.utils import to_categorical

In [30]:
label_map = {label:num for num, label in enumerate(words)}

In [32]:
len(label_map)

29

In [44]:
sequences, labels = [], []
for action in words:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            csv_path = os.path.join(DATA_PATH, action, str(sequence), f'{frame_num}.csv')
            print(f'Loading: {csv_path}')
            res = pd.read_csv(csv_path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

Loading: Landmarked/หนังสือ/0/0.csv
Loading: Landmarked/หนังสือ/0/1.csv
Loading: Landmarked/หนังสือ/0/2.csv
Loading: Landmarked/หนังสือ/0/3.csv
Loading: Landmarked/หนังสือ/0/4.csv
Loading: Landmarked/หนังสือ/0/5.csv
Loading: Landmarked/หนังสือ/0/6.csv
Loading: Landmarked/หนังสือ/0/7.csv
Loading: Landmarked/หนังสือ/0/8.csv
Loading: Landmarked/หนังสือ/0/9.csv
Loading: Landmarked/หนังสือ/0/10.csv
Loading: Landmarked/หนังสือ/0/11.csv
Loading: Landmarked/หนังสือ/0/12.csv
Loading: Landmarked/หนังสือ/0/13.csv
Loading: Landmarked/หนังสือ/0/14.csv
Loading: Landmarked/หนังสือ/0/15.csv
Loading: Landmarked/หนังสือ/0/16.csv
Loading: Landmarked/หนังสือ/0/17.csv
Loading: Landmarked/หนังสือ/0/18.csv
Loading: Landmarked/หนังสือ/0/19.csv
Loading: Landmarked/หนังสือ/0/20.csv
Loading: Landmarked/หนังสือ/0/21.csv
Loading: Landmarked/หนังสือ/0/22.csv
Loading: Landmarked/หนังสือ/0/23.csv
Loading: Landmarked/หนังสือ/0/24.csv
Loading: Landmarked/หนังสือ/0/25.csv
Loading: Landmarked/หนังสือ/0/26.csv
Loading: La

In [45]:
len(sequences)

29

In [46]:
X = np.array(sequences)

In [47]:
X.shape

(29, 30, 3, 34)